In [2]:
import pandas as pd
import datetime
import numpy as np
from tqdm import tqdm_notebook
from datetime import datetime,timedelta
import os
from timeit import default_timer as timer
from scipy import stats
import matplotlib.pyplot as plt
dir_path='/Users/hudsonyeo/Desktop/Python/leo/data/day'
os.chdir('/Users/hudsonyeo/Desktop/Python/leo/data/day')
import time

def process_dir(dir_list):
    x=[]
    for dir_ in dir_list:
        if dir_!='.DS_Store':
            x.append(dir_path+'/'+dir_+'/')
    return x
standard_columns=pd.read_csv('/Users/hudsonyeo/Desktop/Python/leo/data/day/TA/2019.01.02.csv').columns

In [9]:
dir_list_names=os.listdir(dir_path)
dir_list_names.sort()
dir_list=process_dir(dir_list_names)
dir_list 
#list of products available and their directories

['/Users/hudsonyeo/Desktop/Python/leo/data/day/RM/',
 '/Users/hudsonyeo/Desktop/Python/leo/data/day/SR/',
 '/Users/hudsonyeo/Desktop/Python/leo/data/day/TA/',
 '/Users/hudsonyeo/Desktop/Python/leo/data/day/c/',
 '/Users/hudsonyeo/Desktop/Python/leo/data/day/fu/',
 '/Users/hudsonyeo/Desktop/Python/leo/data/day/m/',
 '/Users/hudsonyeo/Desktop/Python/leo/data/day/ni/',
 '/Users/hudsonyeo/Desktop/Python/leo/data/day/zn/']

In [5]:
def get_file_list(dir_path):
    return os.listdir(dir_path)

class categorise():
    def __init__(self):
        self.threshold=[]
        self.percentiles=[25,50,75]
        
    def fit(self,array):
        positive=array[array>0]
        negative=array[array<0]
        self.threshold.append(np.percentile(negative,self.percentiles))   
        self.threshold.append(np.percentile(positive,self.percentiles))
     
    def return_quartile(self,array):
        temp=[]
        for i in array:
            if i>=0:
                if i<self.threshold[1][0]:
                    temp.append(5)
                elif i<self.threshold[1][1]:
                    temp.append(6)
                elif i<self.threshold[1][2]:
                    temp.append(7)
                else:
                    temp.append(8)
            if i<0:
                if i>self.threshold[0][2]:
                    temp.append(4)
                elif i>self.threshold[0][1]:
                    temp.append(3)
                elif i>self.threshold[0][0]:
                    temp.append(2)
                else:
                    temp.append(1)
        return np.asarray(temp)
    
#functions for splitting the data into categories based on their percentiles    
class categorise_simple():
    def __init__(self):
        self.threshold=[]
        self.percentiles=[25,50,75]
        
    def fit(self,array):
        self.threshold.append(np.percentile(array,self.percentiles))
     
    def return_quartile(self,array):
        temp=[]
        for i in array:
                if i<self.threshold[0][0]:
                    temp.append(1)
                elif i<self.threshold[0][1]:
                    temp.append(2)
                elif i<self.threshold[0][2]:
                    temp.append(3)
                else:
                    temp.append(4)
        return np.asarray(temp)    
    
class categorise_10():
    def __init__(self):
        self.threshold=[]
        self.percentiles=[10,20,30,40,50,60,70,80,90]
        
    def fit(self,array):
        positive=array[array>0]
        negative=array[array<0]
        self.threshold.append(np.percentile(negative,self.percentiles))   
        self.threshold.append(np.percentile(positive,self.percentiles))
     
    def return_quartile(self,array):
        temp=[]
        for i in array:
            if i>=0:
                if i<self.threshold[1][0]:
                    temp.append(11)
                elif i<self.threshold[1][1]:
                    temp.append(12)
                elif i<self.threshold[1][2]:
                    temp.append(13)
                elif i<self.threshold[1][3]:
                    temp.append(14)
                elif i<self.threshold[1][4]:
                    temp.append(15)
                elif i<self.threshold[1][5]:
                    temp.append(16)
                elif i<self.threshold[1][6]:
                    temp.append(17)
                elif i<self.threshold[1][7]:
                    temp.append(18)
                elif i<self.threshold[1][8]:
                    temp.append(19)                    
                else:
                    temp.append(20)
            if i<0:
                if i<self.threshold[0][0]:
                    temp.append(1)
                elif i<self.threshold[0][1]:
                    temp.append(2)
                elif i<self.threshold[0][2]:
                    temp.append(3)
                elif i<self.threshold[0][3]:
                    temp.append(4)
                elif i<self.threshold[0][4]:
                    temp.append(5)
                elif i<self.threshold[0][5]:
                    temp.append(6)
                elif i<self.threshold[0][6]:
                    temp.append(7)
                elif i<self.threshold[0][7]:
                    temp.append(8)
                elif i<self.threshold[0][8]:
                    temp.append(9)                    
                else:
                    temp.append(10)
        return np.asarray(temp)    
    
class categorise_x(): #flexible number of categories
    
    def __init__(self,x):
        self.threshold=[]
        self.percentiles=[]
        self.num=x
        for i in range(1,x):
            self.percentiles.append(i*100/x)        
            
    def fit(self,array):
        
        positive=array[array>0]
        negative=array[array<=0]
        self.threshold.append(np.percentile(negative,self.percentiles))   
        self.threshold.append(np.percentile(positive,self.percentiles))
        
    def return_quartile(self,array):
        temp=[]
        for num in array:
            if num<0:
                counter=0
                for i in self.threshold[0]:
                    if num>=i:
                        counter+=1
                    else:
                        break
                temp.append(counter+1)
            else:
                counter=0
                for i in self.threshold[1]:
                    if num>=i:
                        counter+=1
                    else:
                        break
                temp.append(counter+self.num+1)
        return np.asarray(temp)
    
#class for keeping track of time since last cross, cross being positive to negative or reverse    
class cross():
    def __init__(self):
        self.time_last_cross=0
        self.current_sign=True
        self.last_time=datetime(1900, 1, 1, 8, 59)
    def get_time(self,time,price):
        if (time-self.last_time)>timedelta(minutes=1):
            self.last_time=time
            self.time_last_cross=time
            return 0
        self.last_time=time
        if (price>0) and self.current_sign : #if price positive and current trend is also positive
            return (time-self.time_last_cross).total_seconds()
        elif (price<0) and (not self.current_sign): #if price negative and current trend is negative
            return (time-self.time_last_cross).total_seconds()
        else: #if price positive, trend negative or price negative, trend positive
            self.time_last_cross=time
            self.current_sign=(price>0)
            return 0

In [6]:
get_file_list('/Users/hudsonyeo/Desktop/Python/leo/data/day/RM/')

['2019.04.23.csv',
 '2019.01.07.csv',
 '2019.04.22.csv',
 '2019.03.15.csv',
 '2019.04.08.csv',
 '2019.03.01.csv',
 '2019.03.29.csv',
 '2019.01.10.csv',
 '2019.01.04.csv',
 '2019.01.11.csv',
 '2019.03.28.csv',
 '2019.04.09.csv',
 '2019.03.14.csv',
 '2019.04.25.csv',
 '2019.04.19.csv',
 '2019.03.04.csv',
 '2019.01.29.csv',
 '2019.01.15.csv',
 '.DS_Store',
 '2019.01.14.csv',
 '2019.01.28.csv',
 '2019.03.05.csv',
 '2019.04.18.csv',
 '2019.03.11.csv',
 '2019.04.30.csv',
 '2019.04.24.csv',
 '2019.04.26.csv',
 '2019.03.07.csv',
 '2019.03.13.csv',
 '2019.01.02.csv',
 '2019.01.16.csv',
 '2019.01.17.csv',
 '2019.01.03.csv',
 '2019.03.12.csv',
 '2019.03.06.csv',
 '2019.02.21.csv',
 '2019.02.20.csv',
 '2019.02.22.csv',
 '2019.02.27.csv',
 '2019.05.06.csv',
 '2019.05.07.csv',
 '2019.02.26.csv',
 '2019.02.18.csv',
 '2019.02.19.csv',
 '2019.02.25.csv',
 '2019.02.28.csv',
 '2019.02.14.csv',
 '2019.05.09.csv',
 '2019.05.08.csv',
 '2019.02.01.csv',
 '2019.02.15.csv',
 'results',
 '2019.02.12.csv',
 '201

In [7]:
#functions for calculating indicators

def calc_smart_price(dataset):
    data=dataset.copy()
    
    #to combat the limit up event, where price is set to 0. 
    rows=(data.loc[:,'BidPrice1']==0) #count rows of bid price equal 0
    if (np.any(rows)): #if there is such a row
        data.at[rows,'BidPrice1']=data.loc[rows,'AskPrice1'] #for that row, assign ask price to it
    rows=(data.loc[:,'AskPrice1']==0) #do the same for ask price
    if (np.any(rows)):
        data.at[rows,'AskPrice1']=data.loc[rows,'BidPrice1'] 
        
    data['smart_price']=data.loc[:,'BidPrice1']*data.loc[:,'AskVol1']+data.loc[:,'AskPrice1']*data.loc[:,'BidVol1']
    data.at[:,'smart_price']=data.loc[:,'smart_price']/(data.loc[:,['BidVol1','AskVol1']].sum(axis=1))  
    return data

def calc_present_vol(dataset):
    data=dataset.copy()
    data['current_vol']=data.loc[:,'Volume'].diff().fillna(0)/2
    return data

def calc_future_price(dataset,time_ahead=30,time_index=44, price_col=-2):
    data=dataset.copy()
    future_price=[]
    length=len(data)
    for i in range(len(data)):
        current_time=data[i,time_index]+timedelta(seconds=time_ahead)
        
        j=0 #could alternatively use 30 x 3 then search forward and backward
        
        #search forwards
        while((i+j)<length and current_time>data[(i+j),time_index]):
            j+=1
        if (i+j)<length:
            #if index is in the dataframe
            future_price.append(data[(i+j),price_col]) 
        else:
            #price ahead does not exist
            future_price.append(np.nan) 
    future_price=np.asarray(future_price)
    future_price=np.expand_dims(future_price,axis=1)
    return np.concatenate((data,future_price),axis=1)

def calc_edge(dataset,future_col,current_col):
    data=dataset.copy()
    temp=data[:,future_col]-data[:,current_col]
    temp=np.expand_dims(temp,axis=1)
    return np.concatenate((data,temp),axis=1)

def set_index(dataset,time_index=44):
    data=dataset.copy()
    index=data[:,time_index].astype(int)
    new_index=[]
    for j in range(len(index)):
        i=str(index[j]*1000)
        if len(i)==11:
            i='0'+i
        i=i[:-10]+':'+i[-10:]
        i=i[:-8]+':'+i[-8:]
        i=i[:-6]+':'+i[-6:]
        new_index.append(datetime.strptime(i,"%H:%M:%S:%f"))
    data[:,time_index]=new_index
    return data

def calc_sma_fast(dataset,price_col,duration=1,time_index=44): #fast way to calculate SMA, 0.05 seconds for 5000 rows
    data=dataset.copy()
    sma_values=[] 
    smart_sum=np.cumsum(data[:,price_col]) #smart price column is -4
    for i in range(len(data)):
        
        #finding ending point
        last_time=data[i,time_index]-timedelta(minutes=duration)
        
        #finding start point
        j=220*duration#4x60=240
        
        if i-j>0:
            if data[i-j,time_index]>last_time: 
                
                #if starting point time is greater than ending point time
                #search backward
                while(i-j>0 and data[i-j,time_index]>last_time):
                    j+=1
                    
                #activate next line in order to debug and troubleshoot
                #if data[i-j,time_index]!=last_time:
                #    print('backward',i,j,data[i,time_index],data[i-j,time_index],last_time)

                sma=(smart_sum[i]-smart_sum[i-j])/j
                sma_values.append(sma)                
                
            else: 
                
                #search forward
                while(data[i-j,time_index]<last_time):
                    j-=1
                    
                #activate next line in order to debug and troubleshoot
                #if data[i-j,time_index]!=last_time:
                #    print('forward',i,j,data[i,time_index],data[i-j,time_index],last_time)
                
                if j!=0:
                    if data[i-j,time_index]!=last_time:
                        j+=1
                    sma=(smart_sum[i]-smart_sum[i-j])/j
                    sma_values.append(sma)   
                    
                else:
                    sma_values.append(data[i,price_col])
                    
        else: #starting point is at 0
            
            sma=smart_sum[i]/(i+1)
            sma_values.append(sma)                       

    sma_values=np.asarray(sma_values)
    sma_values=data[:,price_col]-sma_values
    sma_values=np.expand_dims(sma_values,axis=1)
    return np.concatenate((data,sma_values),axis=1)  

def calc_volatility_slow(dataset,price_col,duration=15,time_index=44): 
    data=dataset.copy()
    diff=np.diff(dataset[:,price_col])
    diff=np.insert(diff,0,0)
    
    volatility_values=[]
    for i in range(len(data)):
            
        #finding ending point
        last_time=data[i,time_index]-timedelta(minutes=duration)
        
        #finding start point
        j=220*duration#4x60=240
        
        if i-j>0:
            if data[i-j,time_index]>last_time: 
                #if starting point time is greater than ending point time
                #search backward
                while(i-j>0 and data[i-j,time_index]>last_time):
                    j+=1
                vol=np.std(diff[i-j:i])
                volatility_values.append(vol)                     
            else: 
                #search forward
                while(data[i-j,time_index]<last_time):
                    j-=1
                if j!=0:
                    vol=np.std(diff[i-j:i])
                    volatility_values.append(vol) 
                else:
                    volatility_values.append(0)
        else: #starting point is at 0
            if i==0:
                volatility_values.append(0)
                continue
            vol=np.std(diff[:i])
            volatility_values.append(vol)   
            
    volatility_values=np.asarray(volatility_values)
    volatility_values=np.expand_dims(volatility_values,axis=1)
    return np.concatenate((data,volatility_values),axis=1)

def calc_past_vol(dataset,vol_col,duration=1,time_index=44): #
    data=dataset.copy()
    vol_values=[] 
    vol_sum=np.cumsum(data[:,vol_col])
    for i in range(len(data)):
        last_time=data[i,time_index]-timedelta(minutes=duration)
        j=220*duration#4x60=240
        while(i-j>0 and data[i-j,time_index]>last_time):
            j+=1
        if (i-j>=0):
            vol=(vol_sum[i]-vol_sum[i-j])
            vol_values.append(vol)
        else:
            vol=vol_sum[i]
            vol_values.append(vol)
    vol_values=np.asarray(vol_values)
    vol_values=np.expand_dims(vol_values,axis=1)
    return np.concatenate((data,vol_values),axis=1) 

def last_cross(dataset,price_col,time_index=44):
    data=dataset.copy()
    last_cross=cross()
    timings=[]
    for i in range(len(data)):
        timings.append(last_cross.get_time(data[i,time_index],data[i,price_col]))
    timings=np.asarray(timings)
    timings=np.expand_dims(timings,axis=1)
    return np.concatenate((data,timings),axis=1)

def get_case(array):
    current=1
    previous=0
    temp=[]
    for i in array:
        if i==current:
            if i>previous:
                temp.append(1)
            else:
                temp.append(0)
        elif i>current:
            previous=current
            current=i
            temp.append(1)
        else:
            previous=current
            current=i
            temp.append(0)
    return np.asarray(temp)


def binaryToDecimal(binary): 
      
    binary1 = binary 
    decimal, i, n = 0, 0, 0
    while(binary != 0): 
        dec = binary % 10
        decimal = decimal + dec * pow(2, i) 
        binary = binary//10
        i += 1
    return(decimal) 
    
#function to split the data into 4 sets, based on whether the past 2 smartprices have increased or decreased, 2^2=4 
def get_case_n(array,n=2):
    temp=[]
    if n==1:
        return("error")
    previous=np.zeros(n+1)
    for i in array:
        #print(i,previous)
        final=''
        if i==int(previous[-1]):
            before=10
            for j in previous:
                if j>before:
                    final+='1'
                else:
                    final+='0'
                before=int(j)
            final=final[1:]
        else:
            before=10
            for j in previous[1:]:
                if j>before:
                    final+='1'
                else:
                    final+='0'
                before=int(j)
            final=final[1:]
            if i>before:
                final+='1'
            else:
                final+='0'
        
            previous=np.delete(previous,0)
            previous=np.insert(previous,n,i)
        #print(final)
        temp.append(binaryToDecimal(int(final)))
    return np.asarray(temp)    

def process(dataset,sma_duration=1,vol_duration=1,time_index=44):
    data=dataset.copy()
    data=calc_smart_price(data).values #new
    data=set_index(data,time_index=time_index) #no change
    data=calc_future_price(data,time_index=time_index,price_col=-1) #new
    data=calc_edge(data,future_col=-1,current_col=-2) #new
    data=calc_sma_fast(data,duration=sma_duration,time_index=time_index,price_col=-3) #new
    return data

#function to calculate all indicators needed
def process_light(dataset,sma_duration=1,vol_duration=1):
    data=dataset.copy()
    data=calc_smart_price(data)
    data=data.loc[:,['Time','smart_price']].values.astype(object) #new
    data=set_index(data,time_index=0) #no change
    data=calc_future_price(data,time_index=0,price_col=-1) #new
    data=calc_edge(data,future_col=-1,current_col=-2) #new
    data=calc_sma_fast(data,duration=sma_duration,time_index=0,price_col=-3) #new
    return data

In [8]:
#function to process files and return a list of dataframes
def process_files(path,file_list,sma_duration=1):
    df_list=[]
    name_list=[]
    print('processing files...')
    for file in file_list: #read all files and add them to file_list
        if file[-3:]=='csv': #check if file is a CSV
            name_list.append(file)
            df_list.append(process_light(pd.read_csv(path+file),sma_duration=sma_duration))
            print(file)
    print('complete processing')
    return name_list,df_list

In [14]:
#function that analyses the data
#for each day, get the past 20 days of data and get the percentile thresholds
#then split the current day's data into 20, based on the thresholds
#within each split, further split into the 4 cases, which based on the movement of the price
#concatenate all the data for each case into an overall dataframe
#get the mean, and other statistics, and then write to a CSV file
def run_analysis(df_path, df_list,name_list,sma_duration,n=2):

    #create a list to hold all the data
    data_list=[]
    
    #creating a list for each category
    for i in range(20):
        data_list.append([])
        #creating a list for each quartile
        for _ in range((2**n)):
            data_list[i].append([])

    for i in range(len(df_list)): #for each 20 day rolling window
        if i<19: #skip first 19 days
            continue
        print(name_list[i])

        #get -19 day
        sma=df_list[i-19][:,-1].copy() #column for SMA

        #get -18 to 0 day (19 days in total)
        for k in range((i-18),i+1): #get 20 day moving averages
            sma=np.concatenate((sma,df_list[k][:,-1].copy()))

        cat_sma=categorise_10()
        cat_sma.fit(sma) #calculate percentile thresholds for past 20 days

        #get x,y for regression
        x_today=df_list[i][:,-1].copy().astype(float) #column for SMA     
        y_today=df_list[i][:,-2].copy().astype(float) #column for edge

        #removing all NA
        isnum=(~np.isnan(x_today)) & (~np.isnan(y_today))
        x_today=x_today[isnum]
        y_today=y_today[isnum]

        #get categories of today's sma  
        cat_x_today=cat_sma.return_quartile(x_today)

        change=get_case_n(cat_x_today.copy())

        #for each category
        for cat in range(1,21):

            #today's sma filter
            sma_filter_today=(cat_x_today==cat)

            #for each case
            for case in range(2**n):

                filtered= sma_filter_today & (change==case) #filtering by SMA and case
                filtered[:2]=False #removing first 2 entries due to insufficient data
                new_y=y_today[filtered].copy()

                if (len(new_y)!=0):
                    #add today's data into the list by category and case
                    data_list[(cat-1)][case].append(new_y) 

    final_df=pd.DataFrame()                 
    for cat in range(20):
        for case in range(2**n):
            reg_result={}
            if len(data_list[cat][case])==0:
                reg_result['category']=cat+1
                reg_result['case']=case+1
                reg_result['mean']=np.nan
                reg_result['std']=np.nan
                reg_result['num obs']=0
                final_df=final_df.append(reg_result,ignore_index=True)             
                continue

            #getting all the data needed to calculate mean and std
            all_data=data_list[cat][case][0].copy()
            for i in range(1,(len(data_list[cat][case]))):
                all_data=np.concatenate((all_data,data_list[cat][case][i].copy()))

            #adding results to output
            reg_result['category']=cat+1
            reg_result['case']=case+1
            reg_result['mean']=np.mean(all_data)
            reg_result['std']=np.std(all_data)
            reg_result['num obs']=len(all_data)
            final_df=final_df.append(reg_result,ignore_index=True) 

    temp=df_path+'result_10split_'+str(2**n)+'cases_'+str(sma_duration)+'min_sma_weightedmean.csv'
    final_df.to_csv(temp)
    print('done',temp)

In [ ]:
for dir_path in dir_list:
    
    file_list=get_file_list(dir_path)
    file_list.sort()
    
    for sma_duration in [1,3,5,10]:
        start=time.time()
        print('start',dir_path,sma_duration) 

        name_list,df_list=process_files(dir_path, file_list,sma_duration=sma_duration) #process, get SMA and vol data
        mid=time.time()
        print(mid-start)
        #split into categories and get means
        run_analysis(dir_path+'/results/', df_list,name_list,sma_duration=sma_duration)  
        
        end=time.time()
        print('complete',end-mid)

start /Users/hudsonyeo/Desktop/Python/leo/data/day/RM/ 1
processing files...
2019.01.02.csv
2019.01.03.csv
2019.01.04.csv
2019.01.07.csv
2019.01.08.csv
2019.01.09.csv
2019.01.10.csv
2019.01.11.csv
2019.01.14.csv
2019.01.15.csv
2019.01.16.csv
2019.01.17.csv
2019.01.18.csv
2019.01.21.csv
2019.01.22.csv
2019.01.23.csv
2019.01.24.csv
2019.01.25.csv
2019.01.28.csv
2019.01.29.csv
2019.01.30.csv
2019.01.31.csv
2019.02.01.csv
2019.02.11.csv
2019.02.12.csv
2019.02.13.csv
2019.02.14.csv
2019.02.15.csv
2019.02.18.csv
2019.02.19.csv
2019.02.20.csv
2019.02.21.csv
2019.02.22.csv
2019.02.25.csv
2019.02.26.csv
2019.02.27.csv
2019.02.28.csv
2019.03.01.csv
2019.03.04.csv
2019.03.05.csv
2019.03.06.csv
2019.03.07.csv
2019.03.08.csv
2019.03.11.csv
2019.03.12.csv
2019.03.13.csv
2019.03.14.csv
2019.03.15.csv
2019.03.18.csv
2019.03.19.csv
2019.03.20.csv
2019.03.21.csv
2019.03.22.csv
2019.03.25.csv
2019.03.26.csv
2019.03.27.csv
2019.03.28.csv
2019.03.29.csv
2019.04.01.csv
2019.04.02.csv
2019.04.03.csv
2019.04.

In [68]:
error_list

[array([ 8.4277482 ,  8.4153605 ,  8.67371353,  9.71717684,  9.85250664,
        24.12247971, 24.65314282, 24.46319072, 24.86883893, 24.66960762,
        23.49635926, 25.88133768, 25.25488249, 25.37282051]),
 array([25.23011217, 24.69408163, 24.03155726, 25.16373829, 24.94762684,
        25.1653578 , 25.03897334, 24.59804433, 21.01551957, 19.40295815,
        18.3875901 , 17.6774676 , 17.78872549, 17.9068126 , 16.71559252,
        16.60964539, 16.13476314])]

In [3]:
dir_path='/Users/hudsonyeo/Desktop/Python/leo/data/day'
dir_list_names=os.listdir(dir_path)
dir_list_names.sort()
dir_list=process_dir(dir_list_names)
#dir_list=dir_list[4:]
for i in range(len(dir_list)):
    dir_list[i]+='results/result_10sma,6vol_1min,'
dir_list,dir_list_names[1:]

(['/Users/hudsonyeo/Desktop/Python/leo/data/day/RM/results/result_10sma,6vol_1min,',
  '/Users/hudsonyeo/Desktop/Python/leo/data/day/SR/results/result_10sma,6vol_1min,',
  '/Users/hudsonyeo/Desktop/Python/leo/data/day/TA/results/result_10sma,6vol_1min,',
  '/Users/hudsonyeo/Desktop/Python/leo/data/day/c/results/result_10sma,6vol_1min,',
  '/Users/hudsonyeo/Desktop/Python/leo/data/day/fu/results/result_10sma,6vol_1min,',
  '/Users/hudsonyeo/Desktop/Python/leo/data/day/m/results/result_10sma,6vol_1min,',
  '/Users/hudsonyeo/Desktop/Python/leo/data/day/ni/results/result_10sma,6vol_1min,',
  '/Users/hudsonyeo/Desktop/Python/leo/data/day/zn/results/result_10sma,6vol_1min,'],
 ['RM', 'SR', 'TA', 'c', 'fu', 'm', 'ni', 'zn'])

In [17]:
for i in dir_list_names[1:]:
    for num in [1,3,5,10]:
        path='/Users/hudsonyeo/Desktop/Python/leo/data/day/'+i+'/results/result_10sma,6vol_1min,'+str(num)+'min_sma,vol_mean.csv'
        print(i)
        print('sma:',num)
        data=pd.read_csv(path)
        x=[]
        x_std=[]
        pos=[]
        num_obs=[]
        for ma_cat in range(1,21):
            for vol_cat in range(1,5):
                col='ma_cat_'+str(ma_cat)+'_vol_quartile'+str(vol_cat)+'_mean'
                std=data.loc[:,col].std()
                mean=data.loc[:,col].mean()
                column=data.loc[:,col]
                x_std.append(std)
                x.append(mean)
                percentage=len(column[column>=0])/len(column)
                pos.append(percentage)  

                obs='ma_cat_'+str(ma_cat)+'_vol_quartile'+str(vol_cat)+'_num_obs'
                obs=data.loc[:,obs].sum()
                num_obs.append(obs)
        x=np.asarray(x)
        x=np.reshape(x,(20,4))
        x=np.around(x,3)
        x_std=np.asarray(x_std)
        x_std=np.reshape(x_std,(20,4))
        pos=np.asarray(pos)
        pos=np.reshape(pos,(20,4))
        num_obs=np.asarray(num_obs)
        num_obs=np.reshape(num_obs,(20,4))
        print('mean\n',x)
        print('std\n',x_std)
        print('% positive\n',pos)
        print('num obs\n',num_obs)

RM
sma: 1
mean
 [[ 0.094  0.136  0.034  0.328]
 [-0.019  0.001  0.026  0.018]
 [-0.041  0.001  0.007 -0.021]
 [-0.005  0.005  0.009 -0.086]
 [-0.015 -0.01  -0.028 -0.038]
 [-0.05  -0.042 -0.01   0.002]
 [-0.067 -0.042 -0.013 -0.027]
 [-0.114 -0.021 -0.024  0.005]
 [-0.097 -0.085 -0.007 -0.009]
 [-0.045 -0.041 -0.024 -0.011]
 [ 0.049  0.005 -0.017 -0.014]
 [ 0.071  0.039 -0.012 -0.004]
 [ 0.11   0.027  0.001  0.011]
 [ 0.08   0.027  0.026 -0.011]
 [ 0.048 -0.003  0.034 -0.014]
 [ 0.055  0.04   0.01   0.017]
 [ 0.051  0.032  0.042  0.08 ]
 [ 0.014  0.032  0.04   0.083]
 [ 0.08   0.053  0.05   0.081]
 [ 0.048 -0.043  0.008  0.01 ]]
std
 [[0.31192539 0.32511188 0.67743602 0.6643812 ]
 [0.2027036  0.24064907 0.26539329 0.32490333]
 [0.1665734  0.14597644 0.25497499 0.29240182]
 [0.18913937 0.13795886 0.18315377 0.2807386 ]
 [0.21546911 0.14605127 0.14834521 0.22449828]
 [0.13874569 0.14157019 0.18627172 0.22558175]
 [0.11922676 0.1474487  0.12312582 0.19236397]
 [0.1640435  0.17433305 0.136

mean
 [[ 0.178  0.269  0.163  0.241]
 [ 0.075  0.125  0.154  0.089]
 [ 0.014  0.023 -0.001 -0.034]
 [-0.032 -0.016  0.044 -0.036]
 [-0.074 -0.008  0.024 -0.006]
 [-0.008 -0.025 -0.03  -0.022]
 [-0.004 -0.056  0.01  -0.039]
 [-0.029 -0.014 -0.026 -0.   ]
 [-0.019 -0.022  0.009  0.044]
 [-0.011 -0.007  0.013 -0.017]
 [-0.034 -0.019 -0.002  0.03 ]
 [-0.001 -0.009  0.018  0.043]
 [ 0.013  0.041  0.002 -0.005]
 [-0.006  0.035  0.026  0.027]
 [ 0.027 -0.01   0.012  0.088]
 [-0.003  0.016  0.008  0.072]
 [ 0.024  0.016 -0.017  0.116]
 [-0.08   0.008 -0.005  0.028]
 [ 0.063 -0.006 -0.024  0.106]
 [-0.129 -0.108 -0.13  -0.095]]
std
 [[0.68073967 0.63305105 0.65492457 0.60594252]
 [0.36942913 0.29954432 0.3191066  0.46262684]
 [0.33078176 0.29128658 0.36973568 0.3676376 ]
 [0.26953248 0.29823461 0.20595195 0.35128144]
 [0.60565547 0.16575479 0.21946402 0.37214449]
 [0.27044545 0.21569956 0.19369653 0.27645996]
 [0.21881314 0.20053392 0.17695077 0.23769729]
 [0.23139222 0.15903833 0.17195091 0.32

mean
 [[ 0.317  0.589  0.67   0.41 ]
 [ 0.228  0.125  0.039  0.241]
 [ 0.215  0.181  0.168  0.125]
 [-0.109 -0.001  0.071  0.223]
 [-0.079  0.036 -0.018  0.025]
 [ 0.015 -0.053 -0.024 -0.   ]
 [-0.037 -0.103 -0.097  0.031]
 [-0.061  0.004 -0.026 -0.084]
 [-0.237  0.056 -0.05  -0.086]
 [ 0.434 -0.021 -0.027  0.002]
 [ 0.39  -0.029  0.008  0.007]
 [ 0.047 -0.1    0.021  0.   ]
 [ 0.278 -0.047  0.063  0.026]
 [-0.069 -0.032  0.078  0.025]
 [-0.064 -0.072  0.115  0.01 ]
 [-0.065 -0.009  0.01   0.012]
 [-0.106 -0.065  0.002 -0.02 ]
 [-0.253 -0.087  0.095  0.111]
 [-0.225  0.042  0.1    0.162]
 [-0.393 -0.235 -0.231 -0.201]]
std
 [[1.58375024 1.45869657 1.56398873 1.42349318]
 [0.67274059 0.85380618 0.7259584  1.54945806]
 [0.6698353  0.51962377 0.81554407 0.81481102]
 [0.69417593 0.43399066 0.51557175 0.70121158]
 [0.36221211 0.4389501  0.52735428 0.6793954 ]
 [0.51686076 0.29441316 0.38498035 0.62365159]
 [0.3398829  0.44412138 0.33798443 0.53955279]
 [0.3374427  0.29925567 0.41417293 0.49

mean
 [[ 0.972  0.568  0.481  0.793]
 [ 0.238  0.095 -0.088 -0.009]
 [-0.139 -0.104 -0.022  0.149]
 [-0.035 -0.077 -0.151 -0.076]
 [ 0.089  0.009  0.008  0.089]
 [-0.153 -0.128 -0.068  0.032]
 [-0.223 -0.15  -0.114 -0.083]
 [-0.171 -0.068 -0.013 -0.228]
 [ 0.017 -0.024 -0.011  0.073]
 [-0.056  0.003  0.023  0.146]
 [ 0.04  -0.077 -0.053  0.15 ]
 [-0.061  0.062 -0.028 -0.056]
 [ 0.074  0.09   0.147  0.008]
 [ 0.141  0.116  0.112  0.068]
 [ 0.119  0.216  0.204  0.226]
 [-0.049  0.03   0.185  0.33 ]
 [-0.125  0.091  0.14  -0.081]
 [-0.246 -0.187  0.052  0.168]
 [-0.176 -0.169  0.139  0.105]
 [-0.691 -0.67  -0.378 -0.452]]
std
 [[1.61180404 1.64796412 1.8497312  2.80167348]
 [1.11557992 1.16974948 1.26462319 1.59016057]
 [0.98173394 0.86684812 0.8488978  1.12512889]
 [1.10010884 0.77831385 0.94899839 0.91376053]
 [0.6022452  0.62950262 0.80213162 0.93379632]
 [0.62327857 0.77102456 0.70862323 0.91668299]
 [0.93284325 0.76954049 0.77103838 0.88918326]
 [0.73753599 0.46816421 0.61424646 0.87

mean
 [[ 0.093  0.172  0.082  0.138]
 [ 0.032 -0.037 -0.022 -0.016]
 [ 0.034 -0.046  0.016  0.005]
 [-0.012 -0.019 -0.002 -0.026]
 [-0.002  0.013 -0.004 -0.042]
 [ 0.003  0.015 -0.018 -0.038]
 [ 0.012  0.008 -0.029  0.021]
 [ 0.038 -0.01  -0.05   0.006]
 [-0.002  0.016 -0.008  0.006]
 [ 0.031  0.021 -0.012  0.004]
 [ 0.003  0.002  0.003  0.004]
 [ 0.027  0.023  0.018 -0.009]
 [-0.012  0.006  0.01   0.009]
 [-0.002  0.018  0.009  0.003]
 [ 0.014  0.03   0.058  0.013]
 [-0.014 -0.019  0.001  0.025]
 [-0.006 -0.01   0.008  0.058]
 [-0.035 -0.025  0.042 -0.011]
 [-0.014 -0.006  0.07   0.004]
 [-0.124 -0.047  0.009 -0.084]]
std
 [[0.33533926 0.65186704 0.34369055 0.40302841]
 [0.20376121 0.19721266 0.25777001 0.34529434]
 [0.17318899 0.20571398 0.18238508 0.2459847 ]
 [0.13929571 0.1155802  0.17786645 0.16132243]
 [0.2039984  0.12011233 0.11060657 0.13546716]
 [0.14258945 0.0982221  0.14653043 0.15352701]
 [0.15300008 0.15970277 0.15648651 0.16641785]
 [0.21264052 0.12922925 0.12642815 0.17

FileNotFoundError: File b'/Users/hudsonyeo/Desktop/Python/leo/data/day/fu/results/result_10sma,6vol_1min,1min_sma,vol_mean.csv' does not exist

In [64]:
data=pd.read_csv('/Users/hudsonyeo/Desktop/Python/leo/data/day/fu/results/second/result_10sma,6vol_1min,5min_sma,vol_mean.csv')
x=[]
x_std=[]
pos=[]
num_obs=[]
for ma_cat in range(1,21):
    for vol_cat in range(1,5):
        col='ma_cat_'+str(ma_cat)+'_vol_quartile'+str(vol_cat)+'_mean'
        std=data.loc[:,col].std()
        mean=data.loc[:,col].mean()
        column=data.loc[:,col]
        x_std.append(std)
        x.append(mean)
        percentage=len(column[column>=0])/len(column)
        pos.append(percentage)  
        
        obs='ma_cat_'+str(ma_cat)+'_vol_quartile'+str(vol_cat)+'_num_obs'
        obs=data.loc[:,obs].sum()
        num_obs.append(obs)
x=np.asarray(x)
x=np.reshape(x,(20,4))
x=np.around(x,3)
x_std=np.asarray(x_std)
x_std=np.reshape(x_std,(20,4))
pos=np.asarray(pos)
pos=np.reshape(pos,(20,4))
num_obs=np.asarray(num_obs)
num_obs=np.reshape(num_obs,(20,4))
print('mean\n',x)
print('std\n',x_std)
print('% positive\n',pos)
print('num obs\n',num_obs)

mean
 [[ 0.194  0.183  0.285  0.565]
 [ 0.235  0.03  -0.014 -0.072]
 [ 0.103  0.057  0.004 -0.077]
 [ 0.029 -0.003 -0.047  0.091]
 [-0.068  0.03   0.095  0.012]
 [-0.017 -0.05  -0.058 -0.019]
 [-0.085  0.016 -0.073  0.021]
 [-0.029 -0.024 -0.009 -0.034]
 [ 0.001  0.073 -0.04  -0.021]
 [ 0.1    0.006 -0.02  -0.004]
 [ 0.24  -0.029  0.041  0.004]
 [ 0.151  0.08   0.008 -0.105]
 [ 0.024  0.087  0.046 -0.082]
 [ 0.08   0.023 -0.065 -0.166]
 [ 0.018  0.094  0.01  -0.024]
 [ 0.02  -0.014 -0.001 -0.004]
 [ 0.084  0.038 -0.105 -0.023]
 [-0.006  0.066 -0.001 -0.17 ]
 [ 0.125  0.058 -0.01   0.477]
 [-0.257  0.311  0.11  -0.564]]
std
 [[0.93956164 0.87898641 0.78070106 0.88553184]
 [0.51811005 0.47132081 0.66336174 0.8391233 ]
 [0.48218976 0.58471722 0.41383789 0.89423982]
 [0.41440251 0.46260968 0.42024159 0.7358212 ]
 [0.40166118 0.35035442 0.34101024 0.5733473 ]
 [0.37502983 0.42876114 0.44256151 0.58864317]
 [0.33205531 0.2972276  0.41919198 0.56233476]
 [0.31447083 0.42836695 0.38497326 0.53